# Пример использования системы анализа текста

Этот ноутбук демонстрирует работу с текстом, эмбеддингами и их визуализацией.

In [ ]:
from InformationRetrieval.text_embedder import RussianTransformerEmbedder
from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from typing import Iterator, List, Tuple, Union
import numpy as np


In [ ]:
class EnhancedRussianTransformerEmbedder(RussianTransformerEmbedder):
    """Extended version of embedder with support for comparison and visualization."""

    def __init__(self, model_type: str = 'deeppavlov'):
        """Initialize enhanced embedder."""
        super().__init__(model_type=model_type)
        self.umap_reducer = UMAP(n_components=2, random_state=self.DEFAULT_SEED)

    def compare_embeddings(
        self,
        questions: List[str],
        text_chunks: List[str],
    ) -> Tuple[np.ndarray, pd.DataFrame]:
        """Compare embeddings of questions and text.

        Args:
            questions: List of questions
            text_chunks: List of text chunks

        Returns:
            Tuple[np.ndarray, pd.DataFrame]: Similarity matrix and dataframe with results
        """
        question_embeddings = self.model.encode(questions)
        text_embeddings = self.model.encode(text_chunks)
        similarity_matrix = cosine_similarity(question_embeddings, text_embeddings)

        comparison_data = []
        for question_idx, question in enumerate(questions):
            for chunk_idx, chunk in enumerate(text_chunks):
                comparison_data.append(
                    {
                        'Вопрос': question,
                        'Текст': chunk,
                        'Схожесть': similarity_matrix[question_idx][chunk_idx],
                    },
                )

        return similarity_matrix, pd.DataFrame(comparison_data)

    def visualize_embeddings(
        self,
        questions: List[str],
        text_chunks: List[str],
        title: str = 'Визуализация эмбеддингов',
    ) -> go.Figure:
        """Create visualization of embeddings.

        Args:
            questions: List of questions
            text_chunks: List of text chunks
            title: Title of the plot

        Returns:
            go.Figure: Plotly plot
        """
        # Get embeddings
        question_embeddings = self.model.encode(questions)
        text_embeddings = self.model.encode(text_chunks)
        combined_embeddings = np.vstack([question_embeddings, text_embeddings])

        # Reduce dimensionality for visualization
        reduced_embeddings = self.umap_reducer.fit_transform(combined_embeddings)

        # Create labels for visualization
        data_types = ['Вопрос'] * len(questions) + ['Текст'] * len(text_chunks)
        texts = questions + text_chunks

        # Create dataframe for visualization
        visualization_data = pd.DataFrame(
            {
                'x': reduced_embeddings[:, 0],
                'y': reduced_embeddings[:, 1],
                'type': data_types,
                'text': texts,
            },
        )

        return px.scatter(
            visualization_data,
            x='x',
            y='y',
            color='type',
            hover_data=['text'],
            title=title,
        )


In [ ]:
class TextGenerator:
    """Генератор текста с таблицами и списками."""

    def create_sample_text(self) -> str:
        """Создает пример текста с таблицей и списком.

        Returns:
            str: Сгенерированный текст
        """
        sample_text = """
        Отчет о продажах за 2023 год

        ТАБЛИЦА: Квартальные продажи
        Строка 1: Q1 2023 | Продажи: 1000000 | Рост: 15%
        Строка 2: Q2 2023 | Продажи: 1200000 | Рост: 20%
        Строка 3: Q3 2023 | Продажи: 1500000 | Рост: 25%
        Строка 4: Q4 2023 | Продажи: 1800000 | Рост: 30%

        Основные достижения:
        - Увеличение общего объема продаж
        - Расширение клиентской базы
        - Выход на новые рынки

        ТАБЛИЦА: Региональное распределение
        Строка 1: Москва | Доля: 40% | Статус: Активный рост
        Строка 2: Санкт-Петербург | Доля: 30% | Статус: Стабильно
        Строка 3: Регионы | Доля: 30% | Статус: Развитие
        """
        return sample_text

## 1. Генерация примера текста с таблицами

In [ ]:
text_generator = TextGenerator()
sample_text = text_generator.create_sample_text()
print(sample_text)

## 2. Подготовка вопросов и анализ текста

In [ ]:
questions = [
    "Какие продажи были в первом квартале?",
    "Какая доля продаж в Москве?",
    "Какие основные достижения компании?",
    "Как развиваются регионы?"
]

text_chunks = sample_text.split('\n')
text_chunks = [chunk.strip() for chunk in text_chunks if chunk.strip()]

## 3. Создание эмбеддингов и анализ схожести

In [ ]:
embedder = EnhancedRussianTransformerEmbedder()

similarity_matrix, results_df = embedder.compare_embeddings(questions, text_chunks)

for question in questions:
    print(f"\nВопрос: {question}")
    question_results = results_df[results_df['Вопрос'] == question].sort_values('Схожесть', ascending=False).head(3)
    for _, row in question_results.iterrows():
        print(f"Схожесть: {row['Схожесть']:.3f} | Текст: {row['Текст']}")

## 4. Визуализация эмбеддингов

In [ ]:
fig = embedder.visualize_embeddings(questions, text_chunks)
fig.show()